<div align="center">

# Prédiction de la souscription d'un dépôt à terme  

## Pré-traitement des données

<img src="https://raw.githubusercontent.com/komiadok/bank_client_segmentation/main/cover_image.jpg" alt="Segmentation de clients bancaires" width="500"/>

</div>

<div style="background-color:#008080; color:white; padding:15px; border-radius:8px; font-weight:bold; font-size:16px;">
📚 Chargement des librairies
</div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<div style="background-color:#008080; color:white; padding:15px; border-radius:8px; font-weight:bold; font-size:16px;">
🗃️ Partie 1 : Importation des données
</div>

In [2]:
data = pd.read_csv('bank_dataset_explor.csv', sep = ',')

In [3]:
print(f'Taille du dataframe : {data.shape}')

Taille du dataframe : (41188, 25)


In [4]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,False,False,True,False
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,False,False
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,False,False,True,False
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,False,False,False,False
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,False,False,True,False


<div style="background-color:#008080; color:white; padding:15px; border-radius:8px; font-weight:bold; font-size:16px;">
🧹 Partie 2 : Nettoyage des données
</div>

### age

**Rappel**

Lors de l'exploration des données, nous avons détecté cinq individus de moins de 18 ans, donc **mineurs** dans notre jeu de données. Cependant, en raison de leur incapacité juridique à souscrire à un dépôt à terme, nous avons considéré qu'il était plus judicieux de les exclure de l'analyse. En effet, il sont hors de notre scope. C'est une situation impossible niveau opérationnel et réglementaire.

In [5]:
# Création d'une copie du dataframe
clean_data = data.copy()

# Exclusion des mineurs
clean_data = clean_data[clean_data['inconsistent_age'] == False]
print(f"Après exclusion des mineurs, il nous reste {len(clean_data)} enregistrements.")

Après exclusion des mineurs, il nous reste 41183 enregistrements.


### job

**Rappel**

Lors de l’analyse de la variable `job`, nous avons identifié la présence de valeurs manquantes codées sous la modalité `unknown`. Après avoir évalué l’impact de cette modalité sur les souscriptions, nous avons décidé de la conserver. Pour améliorer la clarté de l’analyse, cette modalité sera renommée `missing` afin de la rendre plus explicite et interprétable.

In [6]:
# Renommage de la modalité unknown de job
clean_data['job'] = clean_data['job'].replace('unknown', 'missing')

# Aperçu 
clean_data[clean_data['job'] == 'missing'].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign
29,55,missing,married,university.degree,unknown,unknown,unknown,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False
35,55,missing,married,basic.4y,unknown,yes,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False


### marital

**Rappel**

L'analyse de la variable `job` a mis en évidence des valeurs manquantes codées sous la modalité `unknown`. Bien que cette modalité présente un taux de souscription relativement élevé, son faible effectif ne permet pas d’en tirer une conclusion statistiquement significative. Nous avons néanmoins choisi de la conserver afin de ne pas écarter l’information potentielle qu’elle pourrait apporter à l’analyse. Dans ce contexte, il convient de la renommer en `missing`.

In [7]:
# Renommage de la modalité unknown de marital
clean_data['marital'] = clean_data['marital'].replace('unknown', 'missing')

# Aperçu 
clean_data[clean_data['marital'] == 'missing'].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign
40,58,management,missing,university.degree,no,yes,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False
390,59,retired,missing,university.degree,unknown,no,no,telephone,may,tue,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False


### education

**Rappel**

Lors de l'analyse de la variable `education`, nous avons mis en évidence la présence de la modalité `unknown` qui indiquait les valeurs manquantes. Cette modalité a un effectif non négligeable (**1 731**) et un taux de souscription (**14.50%**) supérieur au taux moyen. C'est donc une modalité **informative**. Il convient donc de la conserver, en la renommant en `missing`.

In [8]:
# Renommage de la modalité unknown de education
clean_data['education'] = clean_data['education'].replace('unknown', 'missing')

# Aperçu 
clean_data[clean_data['education'] == 'missing'].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign
7,41,blue-collar,married,missing,unknown,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False
10,41,blue-collar,married,missing,unknown,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False


### default

**Rappel**

L’exploration des données a révélé la présence de valeurs manquantes dans la variable `default`, codées sous la modalité `unknown`. Ces valeurs, au nombre de **8 597**, présentent un taux de souscription très faible (**5.15%**) comparé au taux moyen. Cette modalité est associée à un comportement spécifique des clients, il est donc judicieux de la conserver en la renommant en `missing`.<br>
Par ailleurs, la modalité `yes` a été identifiée avec un effectif extrêmement faible (**3 occurrences**) et aucun cas de souscription associé. Elle n’est donc pas statistiquement significative et, de ce fait, peu fiable pour notre modèle. Afin d’éviter un risque de surapprentissage, il est recommandé de la supprimer.

In [9]:
# Exclusion de la modalité yes de default
clean_data = clean_data[clean_data['default'] != 'yes']

# Aperçu
print(f"Après exclusion de la modalité yes, il nous reste {len(clean_data)} enregistrements.")

Après exclusion de la modalité yes, il nous reste 41180 enregistrements.


In [10]:
# Renommage de la modalité unknown de default
clean_data['default'] = clean_data['default'].replace('unknown', 'missing')

# Aperçu 
clean_data[clean_data['default'] == 'missing'].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign
1,57,services,married,high.school,missing,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,False,False
5,45,services,married,basic.9y,missing,no,no,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False


### housing

**Rappel**

Lors de l'analyse de la variable `housing`, nous avons identifié trois modalités : `yes`, `no`et `unknown`. Ces modalités affichent des taux de souscription relativement proches, ce qui justifie leur conservation dans l’analyse. Cependant, la modalité `unknown` correspond à des données manquantes ; il est donc pertinent de la renommer en `missing` afin d’en clarifier l’interprétation.

In [11]:
# Renommage de la modalité unknown de housing
clean_data['housing'] = clean_data['housing'].replace('unknown', 'missing')

# Aperçu 
clean_data[clean_data['housing'] == 'missing'].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign
29,55,missing,married,university.degree,missing,missing,unknown,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False
81,51,blue-collar,married,basic.4y,missing,missing,unknown,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False


### loan

**Rappel**

L’analyse de la variable `housing` a révélé la présence de la modalité  `unknown`, correspondant à des valeurs manquantes. Étant donné que son taux de souscription est comparable à celui des autres modalités, il est pertinent de la conserver en la renommant `missing` afin d’en faciliter l’interprétation.

In [12]:
# Renommage de la modalité unknown de loan
clean_data['loan'] = clean_data['loan'].replace('unknown', 'missing')

# Aperçu 
clean_data[clean_data['loan'] == 'missing'].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign
29,55,missing,married,university.degree,missing,missing,missing,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False
81,51,blue-collar,married,basic.4y,missing,missing,missing,telephone,may,mon,...,1.1,93.994,-36.4,4.857,5191.0,no,True,False,True,False


### duration

**Rappel**

L’analyse exploratoire de la variable `duration` a mis en évidence la présence de **29 934 enregistrements** dont la durée d’appel excède le seuil métier et réglementaire fixé à **180 secondes** pour un appel de prospection. Ces valeurs sont considérées comme aberrantes dans la mesure où elles dépassent largement les standards opérationnels, ce qui pourrait indiquer des anomalies de saisie, des appels hors périmètre de la campagne, ou des situations particulières non représentatives du processus standard.<br>
La proportion de ces observations est significative. Une suppression pure et simple risquerait de conduire à :
1. **Une perte d’information** : ces enregistrements peuvent contenir des signaux pertinents sur le comportement ou l’engagement des clients, même s’ils sont atypiques.
2. **Un biais potentiel** : éliminer systématiquement ces données pourrait modifier la distribution des durées et influencer négativement la capacité du modèle à généraliser.

Afin de concilier robustesse statistique et conservation de l’information, nous avons opté pour une stratégie en deux volets :
1. **Winsorisation des valeurs**
   * Les durées supérieures à 180 secondes seront ramenées à ce seuil maximal.
   * Cette transformation permet de limiter l'impact disproportionné des valeurs extrêmes sur les métriques du modèle tout en conservant la donnée dans l'échantillon.
2. **Création d'un indicateur binaire (flag)**
   * Une nouvelle variable sera ajoutée pour signaler au modèle les observations dont la durée réelle excédait initialement 180 secondes.
   * Cet indicateur permettra au modèle d’exploiter la spécificité comportementale de ces cas, tout en travaillant avec des valeurs de duration homogénéisées.

In [13]:
# Plafonner les temps d'appel à 180 secondes
clean_data['duration'] = clean_data['duration'].clip(upper=180)

# Vérification
clean_data[clean_data['duration'] > 180]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign


### campaign

**Rappel**

L’analyse exploratoire de la variable `campaign` a mis en évidence la présence de **3 385 enregistrements** dont le nombre d’appels excède le seuil métier et réglementaire fixé à **5** pour un appel de prospection. Ces valeurs sont considérées comme aberrantes dans la mesure où elles dépassent largement les standards opérationnels, ce qui pourrait indiquer des anomalies de saisie, des appels hors périmètre de la campagne, ou des situations particulières non représentatives du processus standard.<br>
La proportion de ces observations est significative. Une suppression pure et simple risquerait de conduire à :
1. **Une perte d’information** : ces enregistrements peuvent contenir des signaux pertinents sur le comportement ou l’engagement des clients, même s’ils sont atypiques.
2. **Un biais potentiel** : éliminer systématiquement ces données pourrait modifier la distribution des durées et influencer négativement la capacité du modèle à généraliser.

Afin de concilier robustesse statistique et conservation de l’information, nous avons opté pour une stratégie en deux volets :
1. **Winsorisation des valeurs**
   * Les nombres d'appels supérieurs à 5 seront ramenés à ce seuil maximal.
   * Cette transformation permet de limiter l'impact disproportionné des valeurs extrêmes sur les métriques du modèle tout en conservant la donnée dans l'échantillon.
2. **Création d'un indicateur binaire (flag)**
   * Une nouvelle variable sera ajoutée pour signaler au modèle les observations dont le nombre d'appels réel excédait initialement 5.
   * Cet indicateur permettra au modèle d’exploiter la spécificité comportementale de ces cas, tout en travaillant avec des valeurs de duration homogénéisées.

In [14]:
# Plafonner les temps d'appel à 180 secondes
clean_data['campaign'] = clean_data['campaign'].clip(upper=5)

# Vérification
clean_data[clean_data['campaign'] > 5]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign


### pdays

**Rappel**

La variable `pdays` représente le nombre de jours écoulés depuis le dernier contact avec un client. Elle contient la valeur **999**, qui signale que le client n’a jamais été contacté auparavant. Cette valeur extrême, si utilisée telle quelle dans le modèle, risque de perturber la distribution de la variable et d’affecter la qualité des résultats.

Pour pallier ce problème, nous allons procéder de la manière suivante :

* Créer une variable binaire distincte, `never_contacted`, afin d’identifier explicitement les clients qui n’ont jamais été contactés.
* Imputer la valeur médiane de `pdays` (calculée sur les clients déjà contactés) pour remplacer les occurrences de **999** dans la variable continue `pdays`.

Cette approche permettra au modèle de distinguer clairement les clients jamais contactés tout en conservant une variable `pdays` continue et représentative des délais réels de contact.

In [15]:
# Créer la variable binaire indiquant jamais contacté
clean_data['never_contacted'] = (clean_data['pdays'] == 999).astype(int)

# Calculer la médiane des pdays pour les clients déjà contactés
median_pdays = clean_data.loc[clean_data['pdays'] != 999, 'pdays'].median()

# Imputer la médiane pour remplacer les 999
clean_data['pdays'] = clean_data['pdays'].replace(999, median_pdays)

# Aperçu
clean_data[clean_data['never_contacted'] == 1].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign,never_contacted
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,93.994,-36.4,4.857,5191.0,no,False,False,True,False,1
1,57,services,married,high.school,missing,no,no,telephone,may,mon,...,93.994,-36.4,4.857,5191.0,no,True,False,False,False,1


### poutcome

**Rappel**

La variable `poutcome`, représentant le résultat de la dernière campagne marketing, comprend trois modalités distinctes :
* `success` : indique que la campagne précédente a abouti à une souscription réussie.
* `failure` : correspond à un contact effectué lors de la campagne précédente, sans aboutir à une souscription.
* `nonexistent` : signale que le client n’a jamais été contacté lors d’une campagne marketing antérieure.

Pour améliorer la clarté et la compréhension, il est recommandé de renommer la modalité `nonexistent` en `no_previous_contact`. Ce libellé explicite mieux la situation du client et facilite l’interprétation des analyses ultérieures.

In [16]:
# Renommage de la modalité nonexistent de poutcome
clean_data['poutcome'] = clean_data['poutcome'].replace('nonexistent', 'no_previous_contact')

# Aperçu 
clean_data[clean_data['poutcome'] == 'no_previous_contact'].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign,never_contacted
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,93.994,-36.4,4.857,5191.0,no,False,False,True,False,1
1,57,services,married,high.school,missing,no,no,telephone,may,mon,...,93.994,-36.4,4.857,5191.0,no,True,False,False,False,1


<div style="background-color:#008080; color:white; padding:15px; border-radius:8px; font-weight:bold; font-size:16px;">
⚙️ Partie 3 : Feature engineering
</div>

### Création de flag

In [17]:
# Création du flag pour job
clean_data['job_missing_flag'] = (clean_data['job'] == 'missing').astype(int)

# Aperçu
clean_data[clean_data['job_missing_flag'] == 1].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,cons.conf.idx,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign,never_contacted,job_missing_flag
29,55,missing,married,university.degree,missing,missing,missing,telephone,may,mon,...,-36.4,4.857,5191.0,no,True,False,True,False,1,1
35,55,missing,married,basic.4y,missing,yes,no,telephone,may,mon,...,-36.4,4.857,5191.0,no,True,False,True,False,1,1


In [18]:
# Création du flag pour marital
clean_data['marital_missing_flag'] = (clean_data['marital'] == 'missing').astype(int)

# Aperçu
clean_data[clean_data['marital_missing_flag'] == 1].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,euribor3m,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign,never_contacted,job_missing_flag,marital_missing_flag
40,58,management,missing,university.degree,no,yes,no,telephone,may,mon,...,4.857,5191.0,no,True,False,True,False,1,0,1
390,59,retired,missing,university.degree,missing,no,no,telephone,may,tue,...,4.857,5191.0,no,True,False,True,False,1,0,1


In [19]:
# Création du flag pour education
clean_data['education_missing_flag'] = (clean_data['education'] == 'missing').astype(int)

# Aperçu
clean_data[clean_data['education_missing_flag'] == 1].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,nr.employed,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign,never_contacted,job_missing_flag,marital_missing_flag,education_missing_flag
7,41,blue-collar,married,missing,missing,no,no,telephone,may,mon,...,5191.0,no,True,False,True,False,1,0,0,1
10,41,blue-collar,married,missing,missing,no,no,telephone,may,mon,...,5191.0,no,True,False,True,False,1,0,0,1


In [20]:
# Création du flag pour default
clean_data['default_missing_flag'] = (clean_data['default'] == 'missing').astype(int)

# Aperçu
clean_data[clean_data['default_missing_flag'] == 1].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,y,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign,never_contacted,job_missing_flag,marital_missing_flag,education_missing_flag,default_missing_flag
1,57,services,married,high.school,missing,no,no,telephone,may,mon,...,no,True,False,False,False,1,0,0,0,1
5,45,services,married,basic.9y,missing,no,no,telephone,may,mon,...,no,True,False,True,False,1,0,0,0,1


In [21]:
# Création du flag pour housing
clean_data['housing_missing_flag'] = (clean_data['housing'] == 'missing').astype(int)

# Aperçu
clean_data[clean_data['housing_missing_flag'] == 1].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,is_missing,inconsistent_age,is_outlier_duration,is_outlier_campaign,never_contacted,job_missing_flag,marital_missing_flag,education_missing_flag,default_missing_flag,housing_missing_flag
29,55,missing,married,university.degree,missing,missing,missing,telephone,may,mon,...,True,False,True,False,1,1,0,0,1,1
81,51,blue-collar,married,basic.4y,missing,missing,missing,telephone,may,mon,...,True,False,True,False,1,0,0,0,1,1


In [22]:
# Création du flag pour loan
clean_data['loan_missing_flag'] = (clean_data['loan'] == 'missing').astype(int)

# Aperçu
clean_data[clean_data['loan_missing_flag'] == 1].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,inconsistent_age,is_outlier_duration,is_outlier_campaign,never_contacted,job_missing_flag,marital_missing_flag,education_missing_flag,default_missing_flag,housing_missing_flag,loan_missing_flag
29,55,missing,married,university.degree,missing,missing,missing,telephone,may,mon,...,False,True,False,1,1,0,0,1,1,1
81,51,blue-collar,married,basic.4y,missing,missing,missing,telephone,may,mon,...,False,True,False,1,0,0,0,1,1,1


In [23]:
# Création du flag pour duration
clean_data['duration_outlier_flag'] = clean_data['is_outlier_duration'].astype(int)

# Aperçu
clean_data[clean_data['duration_outlier_flag'] == 1].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,is_outlier_duration,is_outlier_campaign,never_contacted,job_missing_flag,marital_missing_flag,education_missing_flag,default_missing_flag,housing_missing_flag,loan_missing_flag,duration_outlier_flag
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,True,False,1,0,0,0,0,0,0,1
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,True,False,1,0,0,0,0,0,0,1


In [24]:
# Création du flag pour duration
clean_data['campaign_outlier_flag'] = clean_data['is_outlier_campaign'].astype(int)

# Aperçu
clean_data[clean_data['campaign_outlier_flag'] == 1].head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,is_outlier_campaign,never_contacted,job_missing_flag,marital_missing_flag,education_missing_flag,default_missing_flag,housing_missing_flag,loan_missing_flag,duration_outlier_flag,campaign_outlier_flag
694,53,admin.,married,university.degree,missing,yes,no,telephone,may,tue,...,True,1,0,0,0,1,0,0,1,1
786,38,blue-collar,married,basic.9y,no,yes,no,telephone,may,wed,...,True,1,0,0,0,0,0,0,1,1


### Création de nouvelles variables

#### Classes d'âge

Pour optimiser l’analyse des données et affiner notre ciblage, nous allons segmenter la population en classes d’âge. Cette segmentation permettra d’identifier plus précisément les segments présentant les plus fortes probabilités de souscription. Par conséquent, elle facilitera la mise en place de communications et de campagnes marketing ciblées, adaptées aux caractéristiques spécifiques de chaque groupe, telles que des offres dédiées aux jeunes actifs.

In [25]:
def assign_age_group(age):
    """
    Description:
        Assigne une catégorie d'âge (groupe d'âge) à une valeur d'âge donnée.

    Arguments:
        age (int ou float): L'âge de la personne.

    Retourne:
        str: La catégorie d'âge correspondante parmi les groupes prédéfinis :
    """

    # Jeunes débutants : généralement étudiants ou jeunes actifs
    if 18 <= age <= 24:
        return 'Jeunes débutants'
    # Jeunes actifs : début de carrière, construction de patrimoine
    elif 25 <= age <= 34:
        return 'Jeunes actifs'
    # Actifs intermédiaires : revenus et responsabilités en progression
    elif 35 <= age <= 44:
        return 'Actifs intermédiaires'
    # Actifs matures : stabilité professionnelle et financière
    elif 45 <= age <= 54:
        return 'Actifs matures'
     # Préretraités : préparation à la retraite, capacité d’épargne accrue
    elif 55 <= age <= 64:
        return 'Préretraités'
    # Retraités récents : capital constitué, recherche de sécurité
    elif 65 <= age <= 74:
        return 'Retraités récents'
    # Retraités avancés : aversion au risque plus marquée, priorité liquidité
    else:
        return 'Retraités avancés'

In [26]:
clean_data['age_group'] = clean_data['age'].apply(assign_age_group)

<div style="background-color:#008080; color:white; padding:15px; border-radius:8px; font-weight:bold; font-size:16px;">
📤 Partie 4 : Exportation des données
</div>

In [27]:
# Affichage de la table à exporter
clean_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,never_contacted,job_missing_flag,marital_missing_flag,education_missing_flag,default_missing_flag,housing_missing_flag,loan_missing_flag,duration_outlier_flag,campaign_outlier_flag,age_group
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,0,0,0,0,0,0,1,0,Préretraités
1,57,services,married,high.school,missing,no,no,telephone,may,mon,...,1,0,0,0,1,0,0,0,0,Préretraités
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,0,0,0,0,0,0,1,0,Actifs intermédiaires
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,0,0,0,0,0,0,0,0,Actifs intermédiaires
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,0,0,0,0,0,0,1,0,Préretraités


In [28]:
clean_data.to_csv("outputs/dataset_preprocess.csv", index=False)